In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

#### change directory if neccessary #################################
MODEL_FOLDER   = "C:/Users/Anita/Documents/UCL/AdvancedTopicsML/Assignment_2/models/lstm_32_d0002_3_layers/"
####################################################################
MODEL_FILENAME = MODEL_FOLDER+"rnn_model__d0002_lstm_3_layers_blaze.ckpt" 

logs_path = 'C:/Users/Anita/Documents/UCL/AdvancedTopicsML/Assignment_2/models/lstm_32_d0002_3_layers'

hm_epochs = 30
n_output = 10
batch_size = 128
chunk_size = 1
n_chunks = 784
rnn_size = 32
n_lin_layer = 100
learning_rate = tf.placeholder('float')
output_keep_prob = tf.placeholder('float')
num_runs = int(mnist.train.num_examples/batch_size)

    

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# threshold function
def binarize(images, threshold=0.1):
    return (threshold < images).astype('float32')

<h4>Define model</h4>

In [3]:
# define placeholders
x = tf.placeholder('float', [None, n_chunks,chunk_size],name='InputData')
y = tf.placeholder('float',name='LabelData')

# define Variables for hidden layers
    
layer1 = {'weights':tf.Variable(tf.random_normal([rnn_size,n_lin_layer]),name='Weights1'),
         'biases':tf.Variable(tf.random_normal([n_lin_layer]),name='Bias1')}
   
layer2 = {'weights':tf.Variable(tf.random_normal([n_lin_layer,n_output]),name='Weights2'),
         'biases':tf.Variable(tf.random_normal([n_output]),name='Bias2')}

# define rnn cell    
lstm_cell = tf.nn.rnn_cell.LSTMCell(rnn_size,state_is_tuple=True)
lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob)
lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells=[lstm_cell] * 3)
outputs, states = tf.nn.dynamic_rnn(cell = lstm_cell, inputs = x,dtype=tf.float32)

# first hidden layer, use only output of last pixel for classification
output = tf.matmul(outputs[:,-1,:],layer1['weights']) + layer1['biases']
nonlin_layer = tf.nn.relu(output)

# second hidden layer
output2 = tf.matmul(nonlin_layer,layer2['weights']) + layer2['biases']


In [4]:
# define loss function, optimizer and accuracy

with tf.name_scope('Model'):
    prediction = output2
with tf.name_scope('Loss'):
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y))
with tf.name_scope('Optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
correct_output = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(correct_output, 'float'))
    
    
# Create summary for loss and accuracy
tf.summary.scalar("loss", loss)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

#init saver
saver = tf.train.Saver()

<h4>Train model </h4>

In [ ]:

with tf.Session() as sess:
    okp = 0.8
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(hm_epochs):
        if (epoch <20):
            lr = 0.0002
        else:
            lr = 0.00005
        for i in range(num_runs): #int(mnist.train.num_examples/batch_size)
            epoch_x, epoch_y = mnist.train.next_batch(batch_size)
            epoch_x = binarize(epoch_x)
            epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))
            _, _ , _, summary = sess.run([optimizer, loss, accuracy,merged_summary_op],
                                           feed_dict={x: epoch_x,y: epoch_y, learning_rate:lr, output_keep_prob:okp })
            summary_writer.add_summary(summary, epoch * num_runs + i)
        
    print('Training finished')        
    saver.save(sess, MODEL_FILENAME)
    print('Model saved') 

<h4>Restore model</h4>

In [6]:
#restore model and calculate testing accuracy and loss 
with tf.Session() as sess:
    okp=1
    lr = 0.0002
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess = sess, save_path= MODEL_FILENAME)
    print("Restored values:")
    loss_test , acc_test = sess.run([ loss, accuracy], feed_dict=
                                       {x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), 
                                        y:mnist.test.labels,learning_rate:lr,output_keep_prob:okp})
    print('Testing Accuracy: %.5f, testing Loss %.5f' %(acc_test,loss_test))
        

Restored values:
Testing Accuracy: 0.75390, testing Loss 0.69015


In [7]:
# restore model and calculate training accuracy and loss 
# devide training set in 10 deterministic batches for easier computation
with tf.Session() as sess:
    okp=1
    lr = 0.0002
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess = sess, save_path= MODEL_FILENAME)
    print("Restored values:")
    loss_train_sum = 0
    acc_train_sum = 0
    for j in range(10):
        loss_train , acc_train = sess.run([loss, accuracy],feed_dict=
                                       {x:mnist.train.images[5500*j:5500*(j+1),:].reshape((-1, n_chunks, chunk_size)),
                                        y:mnist.train.labels[5500*j:5500*(j+1),:],learning_rate:lr,output_keep_prob:okp})
        loss_train_sum += loss_train
        acc_train_sum += acc_train
    loss_train_ave = loss_train_sum/10
    acc_train_ave = acc_train_sum/10
    print('Training Accuracy: %.5f, training Loss: %.5f' %(acc_train_ave,loss_train_ave))

Restored values:
Training Accuracy: 0.74684, training Loss: 0.71088
